In [89]:
import pandas as pd
import numpy as np
import sklearn.metrics.pairwise as pairwise

movies_metadata = pd.read_csv('./movies_metadata.csv')
ratings_small = pd.read_csv('./ratings_small.csv')
keywords = pd.read_csv('./keywords.csv')


/Users/williamcxd7/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [90]:
ratings_small.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [91]:
ratings_small.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int64  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [92]:
ratings_small.describe()

,userId,movieId,rating,timestamp
count,100004.000000,100004.000000,100004.000000,1.000040e+05
mean,347.011310,12548.664363,3.543608,1.129639e+09
std,195.163838,26369.198969,1.058064,1.916858e+08
min,1.000000,1.000000,0.500000,7.896520e+08
25%,182.000000,1028.000000,3.000000,9.658478e+08
50%,367.000000,2406.500000,4.000000,1.110422e+09
75%,520.000000,5418.000000,4.000000,1.296192e+09
max,671.000000,163949.000000,5.000000,1.476641e+09


In [93]:
print(ratings_small.userId.nunique())
print(ratings_small.movieId.nunique()) 

671
9066


In [94]:
ratings_small.duplicated(subset=['userId','movieId']).sum() 
# Thus There is no need for the data cleaning for ratings_small
# We could directly use the data of ratings_small

0

In [95]:
from sklearn.model_selection import train_test_split
train_data,test_data =train_test_split(ratings_small,test_size=0.3) 

In [96]:
m_users = ratings_small.userId.nunique()
n_items = ratings_small.movieId.nunique()

In [97]:
train_data

,userId,movieId,rating,timestamp
97394,652,142240,5.0,1442691006
98019,654,7361,4.0,1145391109
87515,582,2114,2.5,1122167810
81557,555,527,4.0,842283755
89608,596,3481,5.0,1138658587
...,...,...,...,...
47979,353,2516,0.5,1127824651
2999,17,7163,2.0,1127473904
20660,139,5377,4.0,1066860838
40831,294,5305,3.5,1062621868


In [98]:
test_data

,userId,movieId,rating,timestamp
46377,339,54286,3.5,1446663377
34020,243,904,4.0,1118865291
34616,247,1391,4.0,953361200
12150,77,1676,4.0,1183920299
11370,73,53318,4.5,1304411411
...,...,...,...,...
57718,418,5507,3.5,1132696203
10803,73,3574,0.5,1255588780
98662,662,150,4.0,839022109
45624,325,1339,2.5,1356316514


In [99]:
ratings_small_sub = ratings_small.iloc[:,[0,1,2]]

In [100]:
ratings_small_sub

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0
...,...,...,...
99999,671,6268,2.5
100000,671,6269,4.0
100001,671,6365,4.0
100002,671,6385,2.5


In [101]:
# Get ratings matrix in the form of pivot dataframe from the above dataframe
new_ratings_pivot = ratings_small_sub.pivot(index='userId', columns='movieId', values='rating').fillna(0)

In [102]:
# Fill the test data positions with zeros so that the ratings 
# matrix of pivot dataframe contain only train data
new_ratings_pivot_train = new_ratings_pivot.copy()
for userId, movieId,_,_ in test_data.values:
    new_ratings_pivot_train.loc[int(userId),int(movieId)] = 0

In [103]:
# Making it to the matrix form
ratings_small_matrix = np.array(new_ratings_pivot_train)

In [104]:
ratings_small_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [5., 0., 0., ..., 0., 0., 0.]])

In [105]:
# obtain the cosine similarity matrix of each movies
from sklearn.metrics.pairwise import pairwise_distances
item_similarity_m = pairwise_distances(ratings_small_matrix.T,metric='cosine')

In [106]:
item_similarity_m

array([[0.        , 0.67796437, 0.80193114, ..., 1.        , 1.        ,
        0.93250484],
       [0.67796437, 0.        , 0.85551273, ..., 1.        , 1.        ,
        1.        ],
       [0.80193114, 0.85551273, 0.        , ..., 1.        , 1.        ,
        1.        ],
       ...,
       [1.        , 1.        , 1.        , ..., 0.        , 1.        ,
        1.        ],
       [1.        , 1.        , 1.        , ..., 1.        , 0.        ,
        1.        ],
       [0.93250484, 1.        , 1.        , ..., 1.        , 1.        ,
        0.        ]])

In [107]:
# similarly, get the test data matrix by erasing the train data
# in the original data matrix
new_ratings_pivot_test = new_ratings_pivot.copy()
for userId, movieId,_,_ in train_data.values:
    new_ratings_pivot_test.loc[int(userId),int(movieId)] = 0
ratings_small_matrix_test = np.array(new_ratings_pivot_test)

In [108]:
# Get the RMSE of Test Data Set
user_item_prediction = ratings_small_matrix_test.dot(item_similarity_m) / np.array(np.abs(item_similarity_m).sum(axis=1))

from sklearn.metrics import mean_squared_error
from math import sqrt
prediction_flatten = user_item_precdiction[ratings_small_matrix_test.nonzero()]
test_data_matrix_flatten = ratings_small_matrix_test[ratings_small_matrix_test.nonzero()]
error_test = sqrt(mean_squared_error(prediction_flatten,test_data_matrix_flatten))
print('RMSE of Test Data Set',error_test)

RMSE of Test Data Set 3.5805283706688056
